# Final project

Импорты


In [2]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, precision_score, recall_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from googletrans import Translator
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, VotingRegressor, BaggingRegressor, StackingRegressor, VotingClassifier, BaggingClassifier, StackingClassifier
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import average_precision_score
from sklearn.metrics import auc
from sklearn.preprocessing import StandardScaler
from joblib import dump
from urllib import request
import requests
import warnings
from urllib.error import HTTPError, URLError
from tqdm import trange
import tabula

3.1. Прогноз
Подготовка данных

Используйте датасет из Epicurious, подготовленный HugoDarwood
Отфильтруйте поля: удалите как можно больше столбцов, которые не относятся к названиями ингридиентов. 
Вы будете прогнозировать оценку или категорию оценки, используя только ингридиенты и ничего больше. 
Если модель обучится на других данных, которые не будут передаваться на вход, то точность прогнозов будет заведомо ниже.

In [5]:
df = pd.read_csv('C:\\Google Drive\\School21\\Day15\\epi_r.csv')
colum = pd.DataFrame()
colum['name'] = df.columns

In [128]:
#translator = Translator()
#for i in range(0, 200):
#    result = translator.translate(colum.name[i], dest = 'ru')
#   colum['translate'][i] = result.text
#for i in range(200, 400):
#    result = translator.translate(colum.name[i], dest = 'ru')
#    colum['translate'][i] = result.text
#for i in range(400, 600):
#    result = translator.translate(colum.name[i], dest = 'ru')
#    colum['translate'][i] = result.text
#for i in range(600, 601):
#    result = translator.translate(colum.name[i], dest = 'ru')
#    colum['translate'][i] = result.text
#for i in range(601, 681):
#    result = translator.translate(colum.name[i], dest = 'ru')
#    colum['translate'][i] = result.text

In [4]:
#colum.to_excel('C:\\Google Drive\\School21\\Day15\\name_columns_real.xlsx')

In [11]:
df_ingridients = pd.read_excel('C:\\Google Drive\\School21\\Day15\\name_columns.xlsx')
df_ingridients

,Unnamed: 0,name,translate,include,name_real
0,0,title,заглавие,д,title
1,1,rating,рейтинг,д,rating
2,2,calories,калории,NaN,calories
3,3,protein,белок,NaN,protein
4,4,fat,толстый,NaN,fat
...,...,...,...,...,...
675,675,cookbooks,кулинарные книги,NaN,cookbooks
676,676,leftovers,остатки,NaN,leftovers
677,677,snack,перекус,NaN,snack
678,678,snack week,Закуска неделя,NaN,snack week


In [12]:
df_ingr_filtr = df_ingridients['include'].isin(['д'])

In [13]:
df_filtr = df_ingridients.loc[df_ingridients['include'] == 'д']
df_filtr

,Unnamed: 0,name,translate,include,name_real
0,0,title,заглавие,д,title
1,1,rating,рейтинг,д,rating
15,15,almond,миндаль,д,almond
16,16,amaretto,Амаретто,д,amaretto
17,17,anchovy,Анчоус,д,anchovy
...,...,...,...,...,...
662,662,white wine,белое вино,д,white wine
664,664,wild rice,дикий рис,д,wild rice
671,671,yogurt,йогурт,д,yogurt
674,674,zucchini,цуккини,д,zucchini


In [14]:
df_filtr.name_real.unique()

array(['title', 'rating', 'almond', 'amaretto', 'anchovy', 'aperitif',
       'apple', 'apple juice', 'apricot', 'artichoke', 'asian pear',
       'asparagus', 'avocado', 'bacon', 'banana', 'barley', 'basil',
       'bean', 'beef', 'beef rib', 'beef shank', 'beef tenderloin',
       'beer', 'beet', 'bell pepper', 'blackberry', 'blue cheese',
       'blueberry', 'bran', 'brandy', 'breadcrumbs', 'brie', 'brine',
       'brisket', 'broccoli', 'broccoli rabe', 'brown rice',
       'brussel sprout', 'bulgur', 'burrito', 'butter',
       'butterscotch/caramel', 'cabbage', 'cantaloupe', 'capers',
       'caraway', 'cardamom', 'carrot', 'cashew', 'cauliflower', 'celery',
       'champagne', 'chard', 'cheddar', 'cheese', 'cherry', 'chestnut',
       'chicken', 'chickpea', 'chile pepper', 'chili', 'chocolate',
       'cilantro', 'cinnamon', 'clove', 'coconut', 'cod', 'coffee',
       'cognac/armagnac', 'coriander', 'corn', 'cornmeal',
       'cottage cheese', 'couscous', 'crab', 'cranberry',
   

In [15]:
df_head = df[df_filtr.name_real.unique()]

Регрессия

Для прогнозирования рейтинга опробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение на основе gridsearch и кроссвалидации и оцените RMSE на тестовой подвыборке.
Попробуйте различные ансамбли и их гиперпараметры. Выберите лучшее решение на основе gridsearch и кроссвалидации и оцените RMSE на тестовой подвыборке.
Посчитайте RMSE для наивного регрессора, в котором вы просто высчитали средний рейтинг.

In [16]:
X = df_head.drop(columns = ['rating'], axis = 1)
y = df_head['rating']

In [17]:
y

0        2.500
1        4.375
2        3.750
3        5.000
4        3.125
         ...  
20047    3.125
20048    4.375
20049    4.375
20050    4.375
20051    4.375
Name: rating, Length: 20052, dtype: float64

In [18]:
X

,title,almond,amaretto,anchovy,aperitif,apple,apple juice,apricot,artichoke,asian pear,...,vodka,walnut,wasabi,watermelon,whiskey,white wine,wild rice,yogurt,zucchini,turkey
0,"Lentil, Apple, and Turkey Wrap",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20050,Snapper on Angel Hair with Citrus Cream,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [236]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify = y)

# RandomForest

In [237]:
params_rf = {'n_estimators' : [10, 50, 100],
                'max_depth' : [10, 50, 100],
               'max_features' : [10, 50, 100]}
rf = GridSearchCV(RandomForestRegressor(random_state = 21), params_rf, cv = 10, n_jobs = -1)
rf.fit(X_train, y_train)
rf.best_params_

{'max_depth': 50, 'max_features': 10, 'n_estimators': 100}

In [255]:
rf_best = RandomForestRegressor(max_depth = 50, n_estimators = 100, max_features = 10).fit(X_train, y_train)
cross_val_score(rf_best, X_test, y_test, cv = 10, scoring = 'neg_mean_squared_error', n_jobs = -1).mean()

-1.732412225054027

In [244]:
cross_val_score(rf_best, X_test, y_test, cv = 10, n_jobs = -1).mean()

0.033722173425883475

# DisicionTree

In [251]:
tree_params = {'min_samples_split' : [10, 50, 100],
                'max_depth' : [10, 50, 100],
               'max_features' : [10, 50, 100]}
tree = GridSearchCV(DecisionTreeRegressor(), param_grid = tree_params, scoring = 'accuracy', cv = 10, n_jobs = -1)
tree.fit(X_train, y_train)
tree.best_params_

{'max_depth': 10, 'max_features': 10, 'min_samples_split': 10}

In [253]:
tree_best = DecisionTreeRegressor(max_depth = 10, min_samples_split = 10, max_features = 10).fit(X_train, y_train)
cross_val_score(tree_best, X_test, y_test, cv = 10, scoring = 'neg_mean_squared_error', n_jobs = -1).mean()

-1.7926816404945416

In [254]:
cross_val_score(tree_best, X_test, y_test, cv = 10, n_jobs = -1).mean()

-0.004674538420206232

# Voting

In [319]:
vc_params = {'weights' : [(1, 5), (1, 2)]}
vc = GridSearchCV(VotingRegressor(estimators = [('DicisionTreeRegressor', tree_best), ('RandomForestRegressor', rf_best)]), 
                  param_grid = vc_params, scoring = 'accuracy')
vc.fit(X_train, y_train)
vc.best_params_

{'weights': (1, 5)}

In [299]:
vc.score(X_test, y_test)

0.08156706680669212

In [320]:
vc_best = VotingRegressor(estimators = [('DicisionTreeRegressor', tree_best), ('RandomForestRegressor', rf_best)], weights = [1, 5]).fit(X_train, y_train)
cross_val_score(vc_best, X_test, y_test, cv = 10, scoring = 'neg_mean_squared_error', n_jobs = -1).mean()

-1.7205653785065507

# Bagging

In [331]:
bag_params = {'n_estimators' : [10, 20, 30]}
bag = GridSearchCV(BaggingRegressor(base_estimator = tree_best), 
                  param_grid = bag_params, scoring = 'accuracy')
bag.fit(X_train, y_train)
bag.best_params_

{'n_estimators': 10}

In [336]:
bag_best = BaggingRegressor(base_estimator = tree_best, n_estimators = 10)
cross_val_score(bag_best, X_test, y_test, cv = 10, scoring = 'neg_mean_squared_error', n_jobs = -1).mean()

-1.730099803672554

In [338]:
bag_params = {'n_estimators' : [10, 20, 30]}
bag = GridSearchCV(BaggingRegressor(base_estimator = rf_best), 
                  param_grid = bag_params, scoring = 'accuracy')
bag.fit(X_train, y_train)
bag.best_params_

{'n_estimators': 10}

In [340]:
bag_best = BaggingRegressor(base_estimator = rf_best, n_estimators = 10)
cross_val_score(bag_best, X_test, y_test, cv = 10, scoring = 'neg_mean_squared_error', n_jobs = -1).mean()

-1.7099153045163586

# Stacking

In [348]:
st_params = {'passthrough' : [True, False]}
st = GridSearchCV(StackingRegressor(estimators = [('DicisionTreeRegressor', tree_best), ('RandomForestRegressor', rf_best)]), 
                  param_grid = st_params, scoring = 'accuracy')
st.fit(X_train, y_train)
st.best_params_

{'passthrough': True}

In [349]:
st_best = StackingRegressor(estimators = [('DicisionTreeRegressor', tree_best), ('RandomForestRegressor', rf_best)], passthrough = True)
cross_val_score(st_best, X_test, y_test, cv = 10, scoring = 'neg_mean_squared_error', n_jobs = -1).mean()

-1.7153301944075963

# Naiv classifier

In [339]:
y_mean = y_test.mean()
y_mean

3.7146285215656945

In [352]:
y_pred = [y_mean for i in range(len(y_test))]
math.sqrt(mean_squared_error(y_pred, y_test))

1.3406047250383304

# Классификация

Бинаризируйте значения целевой переменной путем округления рейтингов до ближайшего целого числа. Это и будут ваши классы.
Для прогнозирования классов попробуйте различные алгоритмы и их гиперпараметры. 
Выберите лучшее решение на основе gridsearch и кроссвалидации и посчитайте accuracy на тестовой подвыборке.
Рассчитайте значение accuracy наивного классификатора, в котором вы для всех наблюдений прогнозируете наиболее распространенный класс. 
Сравните метрики.

In [354]:
y = round(y, 0).astype(int)
y.unique()

array([2, 4, 5, 3, 0, 1])

In [355]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify = y)

# DicisionTree

In [356]:
param_grid = {'max_depth': range(1, 20),
             'class_weight' : ['balanced', None],
             'criterion' : ['entropy', 'gini']}

In [357]:
grid = GridSearchCV(estimator = DecisionTreeClassifier(random_state=21), param_grid = param_grid)
grid.fit(X_train, y_train)
grid.best_score_

0.6633625697364567

In [ ]:
grid.best_params_

In [358]:
cross_val_score(grid, X_test, y_test, cv = 10, scoring = 'neg_mean_squared_error', n_jobs = -1).mean()

-1.8220096524857012

# RandomForest

In [360]:
param_rf = {'n_estimators': [5, 10, 30, 50],
              'max_depth' : range(1, 50),
             'class_weight' : ['balanced', None],
             'criterion' : ['entropy', 'gini']}
rf_grid = GridSearchCV(estimator = RandomForestClassifier(random_state=21), param_grid = param_rf)
rf_grid.fit(X_train, y_train)
rf_grid.best_score_

0.6676638685306054

In [361]:
rf_grid.best_params_

{'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 35,
 'n_estimators': 30}

In [362]:
cross_val_score(rf_grid, X_test, y_test, cv = 10, scoring = 'neg_mean_squared_error', n_jobs = -1).mean()

-1.8439746405131452

# LogisticRegression

In [363]:
param_lr = {'penalty': ['l2', None],
              'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
             'fit_intercept' : [True, False]}
lr_grid = GridSearchCV(estimator = LogisticRegression(random_state=21), param_grid = param_lr)
lr_grid.fit(X_train, y_train)
lr_grid.best_score_

0.6662298756070248

In [364]:
lr_grid.best_params_

{'fit_intercept': True, 'penalty': 'l2', 'solver': 'newton-cg'}

In [365]:
cross_val_score(lr_grid, X_test, y_test, cv = 10, scoring = 'neg_mean_squared_error', n_jobs = -1).mean()

-1.952370317986129

In [366]:
cross_val_score(lr_grid, X_test, y_test, cv = 10, n_jobs = -1).mean()

0.6629278793067084

In [384]:
y_mean = y_test.mean()
y_mean = round(y_mean)

In [385]:
y_pred = [y_mean for i in range(len(y_test))]
y_pred

[4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,


In [375]:
best_model = RandomForestClassifier(class_weight = None, criterion = 'entropy', max_depth = 35, n_estimators = 30, random_state=21)
best_model.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=35, n_estimators=30,
                       random_state=21)

In [386]:
accuracy_score(y_pred, best_model.predict(X_test))

0.9586138120169534

Снова проведите бинаризацию целевого столбца, преобразовав теперь целые числа в классы bad (0, 1) (невкусное), so-so (2, 3) (нормальное), great (4, 5) (вкусное).
Снова для прогнозирования классов попробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение на основе кроссвалидации и посчитайте accuracy на тестовой подвыборке.
Снова рассчитайте значение accuracy наивного классификатора, в котором вы для всех наблюдений прогнозируете наиболее распространенный класс. Сравните метрики.
Что хуже: спрогнозировать плохой рейтинг, который на самом деле окажется хорошим, или спрогнозировать хороший рейтинг, который на самом деле окажется плохим?

In [389]:
target = y.copy()
target = target.replace(0, 'bad')
target = target.replace(1, 'bad')
target = target.replace(2, 'so-so')
target = target.replace(3, 'so-so')
target = target.replace(4, 'great')
target = target.replace(5, 'great')

In [390]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=21, stratify = target)

# DicisionTree

In [391]:
param_tree = {'max_depth': range(1, 20),
             'class_weight' : ['balanced', None],
             'criterion' : ['entropy', 'gini']}
tree_grid = GridSearchCV(estimator = DecisionTreeClassifier(random_state=21), param_grid = param_tree)
tree_grid.fit(X_train, y_train)
tree_grid.best_score_

0.7985785186457354

In [392]:
tree_grid.best_params_

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 1}

In [393]:
cross_val_score(tree_grid, X_test, y_test, cv = 10, n_jobs = -1).mean()

0.7955614694606767

# RandomForest

In [394]:
param_rf = {'n_estimators': [5, 10, 30, 50],
              'max_depth' : range(1, 50),
             'class_weight' : ['balanced', None],
             'criterion' : ['entropy', 'gini']}
rf_grid = GridSearchCV(estimator = RandomForestClassifier(random_state=21), param_grid = param_rf)
rf_grid.fit(X_train, y_train)
rf_grid.best_score_

0.8005110509795937

In [395]:
rf_grid.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 43,
 'n_estimators': 30}

In [396]:
cross_val_score(rf_grid, X_test, y_test, cv = 10, n_jobs = -1).mean()

0.7955620898003748

# LogisticRegression

In [397]:
param_lr = {'penalty': ['l2', None],
              'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
             'fit_intercept' : [True, False]}
lr_grid = GridSearchCV(estimator = LogisticRegression(random_state=21), param_grid = param_lr)
lr_grid.fit(X_train, y_train)
lr_grid.best_score_

0.7987033234924531

In [398]:
lr_grid.best_params_

{'fit_intercept': True, 'penalty': 'l2', 'solver': 'lbfgs'}

In [399]:
cross_val_score(lr_grid, X_test, y_test, cv = 10, n_jobs = -1).mean()

0.7930664631952457

# Naive Regressor

In [401]:
best_model = RandomForestClassifier(class_weight = None, criterion = 'gini', max_depth = 43, n_estimators = 30, random_state=21)
best_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=43, n_estimators=30, random_state=21)

In [403]:
y_mean = 'great'
y_pred = [y_mean for i in range(len(y_test))]
y_pred

['great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',
 'great',


In [404]:
accuracy_score(y_pred, best_model.predict(X_test))

0.9693343305908751

Замените метрику accuracy другой соответствующей метрикой.
Для прогнозирования классов с новой метрикой опробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение и посчитайте метрику на тестовой подвыборке.

In [409]:
y_pred = best_model.predict(X_train)
y_pred_proba = best_model.predict_proba(X_train)

In [412]:
print(f'accuracy is {accuracy_score(y_train, y_pred):.5f}\nprecision is {precision_score(y_train, y_pred, average = "weighted"):.5f}\nrecall is {recall_score(y_train, y_pred,average = "weighted"):.5f}')

accuracy is 0.84222
precision is 0.86303
recall is 0.84222


Best metrics - precision

# LogisticRegression

In [418]:
param_lr = {'penalty': ['l2', None],
              'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
             'fit_intercept' : [True, False]}
lr_grid = GridSearchCV(estimator = LogisticRegression(random_state=21), param_grid = param_lr, scoring = 'precision')
lr_grid.fit(X_train, y_train)
lr_grid.best_params_

{'fit_intercept': True, 'penalty': 'l2', 'solver': 'newton-cg'}

In [421]:
lr_best = LogisticRegression(penalty = 'l2', solver = 'newton-cg', fit_intercept = True, random_state = 21).fit(X_train, y_train)

In [422]:
precision_score(y_test, lr_best.predict(X_test), average = 'weighted')

0.7468670053559037

# DicisionTree

In [423]:
param_tree = {'max_depth': range(1, 20),
             'class_weight' : ['balanced', None],
             'criterion' : ['entropy', 'gini']}
tree_grid = GridSearchCV(estimator = DecisionTreeClassifier(random_state=21), param_grid = param_tree, scoring = 'precision')
tree_grid.fit(X_train, y_train)
tree_grid.best_params_

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 1}

In [425]:
dt_best = DecisionTreeClassifier(max_depth = 1, class_weight = 'balanced', criterion = 'entropy', random_state = 21).fit(X_train, y_train)

In [426]:
precision_score(y_test, dt_best.predict(X_test), average = 'weighted')

0.07471759903614572

# RandomForest

In [427]:
param_rf = {'n_estimators': [5, 10, 30, 50],
              'max_depth' : range(1, 50),
             'class_weight' : ['balanced', None],
             'criterion' : ['entropy', 'gini']}
rf_grid = GridSearchCV(estimator = RandomForestClassifier(random_state=21), param_grid = param_rf)
rf_grid.fit(X_train, y_train)
rf_grid.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 43,
 'n_estimators': 30}

In [430]:
rf_best = RandomForestClassifier(max_depth = 43, class_weight = None, criterion = 'gini', n_estimators = 30, random_state = 21).fit(X_train, y_train)

In [431]:
precision_score(y_test, rf_best.predict(X_test), average = 'weighted')

0.759732740242558

Попробуйте различные ансамбли и их гиперпараметры. Выберите лучшее решение и посчитайте метрику на тестовой подвыборке.

# Voting

In [469]:
vc_params = {'weights' : [(1, 2, 3), (3, 2, 1), (1, 3, 2)]}
vc = GridSearchCV(VotingClassifier(estimators = [('DicisionTreeClassifier', dt_best), ('RandomForestClassifier', rf_best), ('LogisticRegression', lr_best)]), 
                  param_grid = vc_params, scoring = 'precision')
vc.fit(X_train, y_train)
vc.best_params_

{'weights': (1, 2, 3)}

In [455]:
vc_best = VotingClassifier(estimators = [('DicisionTreeClassifier', dt_best), ('RandomForestClassifier', rf_best), ('LogisticRegression', lr_best)], weights = [1, 2, 5]).fit(X_train, y_train)
precision_score(y_test, vc_best.predict(X_test), average = 'weighted')

0.7468670053559037

# Bagging

In [438]:
bag_params = {'n_estimators' : [10, 20, 30]}
bag = GridSearchCV(BaggingClassifier(base_estimator = dt_best), 
                  param_grid = bag_params, scoring = 'precision')
bag.fit(X_train, y_train)
bag.best_params_

{'n_estimators': 10}

In [456]:
bag_best_dt = BaggingClassifier(base_estimator = dt_best, n_estimators = 10)
bag_best_dt.fit(X_train, y_train)
print('Precision for DecisionTree', precision_score(y_test, bag_best_dt.predict(X_test), average = 'weighted'))

Precision for DecisionTree 0.07471759903614572


In [439]:
bag_params = {'n_estimators' : [10, 20, 30]}
bag = GridSearchCV(BaggingClassifier(base_estimator = rf_best), 
                  param_grid = bag_params, scoring = 'precision')
bag.fit(X_train, y_train)
bag.best_params_

{'n_estimators': 10}

In [457]:
bag_best_rf = BaggingClassifier(base_estimator = rf_best, n_estimators = 10).fit(X_train, y_train)
print('Precision for RandomForest', precision_score(y_test, bag_best_rf.predict(X_test), average = 'weighted'))

Precision for RandomForest 0.7716713088580496


# Stacking

In [446]:
st_params = {'passthrough' : [True, False]}
st = GridSearchCV(StackingClassifier(estimators = [('DicisionTreeClassifier', dt_best), ('RandomForestClassifier', rf_best), ('LogisticRegression', lr_best)]), 
                  param_grid = st_params, scoring = 'precision')
st.fit(X_train, y_train)
st.best_params_

{'passthrough': True}

In [460]:
st_best = StackingClassifier(estimators = [('DicisionTreeClassifier', dt_best), ('RandomForestClassifier', rf_best), ('LogisticRegression', lr_best)], passthrough = True).fit(X_train, y_train)
print('Precision for Stacking', precision_score(y_test, st_best.predict(X_test), average = 'weighted'))

Precision for RandomForest 0.8033541748003632


In [470]:
final_model = st_best = StackingClassifier(estimators = [('DicisionTreeClassifier', dt_best), ('RandomForestClassifier', rf_best), ('LogisticRegression', lr_best)], passthrough = True).fit(X_train, y_train)

In [475]:
dump(final_model, 'C:\\Google Drive\\School21\\Day15\\best_model.joblib')

['C:\\Google Drive\\School21\\Day15\\best_model.joblib']